In [1]:
path2datadir <- "/home/xinyuelu/AD-proj/marker_genes"

## Read 10x whole brain markers

In [ ]:
data <- read.csv("/home/xinyuelu/AD-proj/marker_genes/Xenium_hBrain_v1_metadata.csv")
head(data)
unique_annotations <- unique(data$Annotation)
exc_subtypes <- sort(unique_annotations[grep("^L", unique_annotations)])
exc_subtypes <- exc_subtypes[-((length(exc_subtypes) - 1):length(exc_subtypes))]
marker_list <- list()
for (subtype in exc_subtypes) {
  print(subtype)
  tmp <- data[data$Annotation == subtype, ]['Genes']
  print(tmp)
  marker_list[[subtype]] <- tmp
}

,Genes,Ensembl_ID,Num_Probesets,Codewords,Annotation
,<chr>,<chr>,<int>,<int>,<chr>
1,ABCC9,ENSG00000069431,8,1,VLMC
2,ADAMTS12,ENSG00000151388,8,1,VLMC
3,ADAMTS16,ENSG00000145536,8,1,L4 IT
4,ADAMTS3,ENSG00000156140,8,1,L6 IT
5,ADRA1A,ENSG00000120907,8,1,Sncg
6,ADRA1B,ENSG00000170214,8,1,Sncg


In [12]:
exc_subtypes

[1] "L2/3 IT"    "L4 IT"      "L5 ET"      "L5 IT"      "L5/6 NP"   
[6] "L6 CT"      "L6 IT"      "L6 IT Car3" "L6b"

In [5]:
unique_annotations <- unique(data$Annotation)
unique_annotations

[1] "VLMC"                        "L4 IT"                      
 [3] "L6 IT"                       "Sncg"                       
 [5] "Glioblastoma (TME)"          "Chandelier"                 
 [7] "Pax6"                        "Sst Chodl"                  
 [9] "Glioblastoma (Cancer cells)" "Microglia-PVM"              
[11] "Broad"                       "Astrocyte"                  
[13] "L5/6 NP"                     "OPC"                        
[15] "Pvalb"                       "Oligodendrocyte"            
[17] "Proliferation"               "L6 IT Car3"                 
[19] "Endothelial"                 "Sst"                        
[21] "L2/3 IT"                     "Lamp5"                      
[23] "Lamp5 Lhx6"                  "L6 CT"                      
[25] "L5 IT"                       "L6b"                        
[27] "L5 ET"                       "Vip"

In [11]:
minor_subtypes <- c("VLMC", "Glioblastoma (TME)", 
"Chandelier", "Pax6", "Glioblastoma (Cancer cells)", 
"Microglia-PVM")


In [7]:
# Extract Exc subtypes
exc_subtypes <- sort(unique_annotations[grep("^L", unique_annotations)])
exc_subtypes <- exc_subtypes[-((length(exc_subtypes) - 1):length(exc_subtypes))]
exc_subtypes

[1] "L2/3 IT"    "L4 IT"      "L5 ET"      "L5 IT"      "L5/6 NP"   
[6] "L6 CT"      "L6 IT"      "L6 IT Car3" "L6b"

In [10]:
marker_list <- list()
for (subtype in exc_subtypes) {
  print(subtype)
  tmp <- data[data$Annotation == subtype, ]['Genes']
  print(tmp)
  marker_list[[subtype]] <- tmp
}

[1] "L2/3 IT"
     Genes
67    CUX2
238 TESPA1
265   ZBBX
[1] "L4 IT"
       Genes
3   ADAMTS16
90     FSTL4
169    OTOGL
184   POU6F2
201     RORB
212  SLC17A6
[1] "L5 ET"
     Genes
174  PCSK1
198   RIT2
216  SLIT3
217   SNCG
219 SORCS1
[1] "L5 IT"
    Genes
107 HTR2A
204 RXFP1
[1] "L5/6 NP"
     Genes
18  ATP2C2
37    CD36
62    CRYM
108  HTR2C
250   TPH2
257  TSHZ2
263  VWC2L
[1] "L6 CT"
     Genes
88  FILIP1
105 HS3ST2
106 HS3ST4
121  KCNH5
[1] "L6 IT"
      Genes
4   ADAMTS3
23    C1QL3
218   SNTB2
[1] "L6 IT Car3"
      Genes
46    CDH12
93   GAS2L3
158   NPY1R
163   NTNG2
164    NWD2
183   POSTN
193 RASGRP1
215 SLC26A4
243  THEMIS
255   TRPC5
266 ZDHHC23
[1] "L6b"
      Genes
156  NPFFR2
202    ROS1
213 SLC17A7


## Read Alien Brain called markers

In [1]:
path2data <- "/home/xinyuelu/AD-proj/marker_genes/ensembl"
path2res <- "/home/xinyuelu/AD-proj/marker_genes/gene_symbol"

In [2]:
path2genemapping <- "/home/xinyuelu/AD-proj/scripts_results/annotate_cells/gene_mapping.csv"

In [5]:
data <- read.csv(path2genemapping, row.names = 1)

In [7]:
gene_map <- setNames(data$gene_name, data$gene_id)

In [8]:
marker_files <- list.files(path2data, pattern = "\\.csv$", full.names = TRUE)

In [16]:
final_markers <- list()
unique_markers <- list()
for (file in marker_files) {
  
  data <- read.csv(file)
  
  if (!"names" %in% colnames(data)) {
    warning(paste("The file", file, "does not have a 'names' column. Skipping."))
    next
  }

  mapping_vector <- data$names
  top_30_markers <- mapping_vector[1:30]
  top_30_markers <- gene_map[top_30_markers]
  top_30_markers <- top_30_markers[!is.na(top_30_markers)]

  top_markers <- setdiff(top_30_markers, unlist(unique_markers))
  unique_markers<- append(unique_markers, list(top_markers))
  cell_type_name <- basename(file)
  cell_type_name <- strsplit(cell_type_name, "_")[[1]][1]
  final_markers[[cell_type_name]] <- head(top_markers, 10)

  
  # Create an output file name (change extension to .txt)
  output_file <- sub("\\.csv$", ".txt", file)
  output_file <- file.path(path2res, basename(output_file))
  
  # Write the first 30 elements to the output file, each on a new line
  writeLines(top_30_markers, output_file)
  
  message(paste("Processed file:", file, "and saved to:", output_file))
}

Processed file: /home/xinyuelu/AD-proj/marker_genes/ensembl/Astrocyte_markers.csv and saved to: /home/xinyuelu/AD-proj/marker_genes/gene_symbol/Astrocyte_markers.txt

Processed file: /home/xinyuelu/AD-proj/marker_genes/ensembl/Chandelier_markers.csv and saved to: /home/xinyuelu/AD-proj/marker_genes/gene_symbol/Chandelier_markers.txt

Processed file: /home/xinyuelu/AD-proj/marker_genes/ensembl/Endothelial_markers.csv and saved to: /home/xinyuelu/AD-proj/marker_genes/gene_symbol/Endothelial_markers.txt

Processed file: /home/xinyuelu/AD-proj/marker_genes/ensembl/L2-3 IT_markers.csv and saved to: /home/xinyuelu/AD-proj/marker_genes/gene_symbol/L2-3 IT_markers.txt

Processed file: /home/xinyuelu/AD-proj/marker_genes/ensembl/L4 IT_markers.csv and saved to: /home/xinyuelu/AD-proj/marker_genes/gene_symbol/L4 IT_markers.txt

Processed file: /home/xinyuelu/AD-proj/marker_genes/ensembl/L5 ET_markers.csv and saved to: /home/xinyuelu/AD-proj/marker_genes/gene_symbol/L5 ET_markers.txt

Processed fi

In [17]:
final_markers

$Astrocyte
 [1] "LSAMP"  "NEAT1"  "NPAS3"  "SLC1A2" "CTNNA2" "GPM6A"  "PCDH9"  "DTNA"  
 [9] "GPC5"   "ADGRV1"

$Chandelier
 [1] "CNTN5"   "ERBB4"   "ZNF385D" "GALNTL6" "RBFOX1"  "ADGRB3"  "THSD7A" 
 [8] "DLGAP2"  "CNTNAP2" "LRRC4C" 

$Endothelial
 [1] "ST6GALNAC3" "FLT1"       "PTPRG"      "MT2A"       "ANO2"      
 [6] "ABCB1"      "RBMS3"      "ATP10A"     "MECOM"      "ELOVL7"    

$`L2-3 IT`
 [1] "KCNIP4" "HS6ST3" "CDH18"  "DGKB"   "LRRTM4" "PTPRD"  "PDZD2"  "SYN3"  
 [9] "OPCML"  "ATRNL1"

$`L4 IT`
 [1] "PLCH1"  "TSHZ2"  "PTPRT"  "GRM7"   "GUSBP5" "RMST"   "DCC"    "LINGO2"
 [9] "KCND2"  "KCNQ5" 

$`L5 ET`
 [1] "ROBO2"  "GRIK2"  "NRG1"   "NALF1"  "ASIC2"  "NEGR1"  "CCSER1" "AGBL4" 
 [9] "PEX5L"  "RYR2"  

$`L5 IT`
 [1] "PTPRK"           "RALYL"           "TOX"             "ENSG00000228655"
 [5] "PDE1A"           "PHACTR1"         "CASC15"          "CADPS2"         
 [9] "IL1RAPL2"        "ENSG00000254394"

$`L5-6 NP`
 [1] "SNTG1"           "NLGN1"           "NRXN1"           "ENSG00000283674"
 [5] "NAV3"            "HTR2C"           "SNHG14"          "DAB1"           
 [9] "MAPK10"          "KHDRBS2"        

$`L6 CT`
 [1] "CSMD1"    "TRPM3"    "PDE4D"    "PDZRN4"   "ANKS1B"   "ADAMTSL1"
 [7] "HS3ST4"   "KIAA1217" "SLC8A1"   "ZNF385B" 

$`L6 IT Car3`
 [1] "ERC2"    "KCTD16"  "ZNF804B" "UNC5C"   "CADPS"   "SORBS2"  "FRAS1"  
 [8] "SYNPR"   "MMP16"   "AK5"    

$`L6 IT`
[1] "NCAM2"  "ADGRL3" "MCTP1"  "EFNA5"  "CDH13"  "CELF2"  "CNTN4"  "LRRC7" 

$L6b
[1] "SGCZ"    "GRM5"    "KALRN"   "ZFHX3"   "XKR4"    "MGAT4C"  "PCDH11X"
[8] "CHRM3"  

$`Lamp5 Lhx6`
 [1] "UNC5D"    "GRIP1"    "KAZN"     "ADARB2"   "FGF13"    "NXPH1"   
 [7] "FGF14"    "KIRREL3"  "CACNA2D1" "NTNG1"   

$Lamp5
[1] "RELN"   "GRIK1"  "GRIN2A" "INPP4B" "PLCB1"  "CSMD3"  "FBXL7"  "SLC6A1"

$`Microglia-PVM`
 [1] "PLXDC2"   "LRMDA"    "DOCK4"    "ARHGAP24" "FRMD4A"   "CHST11"  
 [7] "SPP1"     "ELMO1"    "SSH2"     "LDLRAD4" 

$Oligodendrocyte
 [1] "CTNNA3"  "ST18"    "PLP1"    "QKI"     "PDE4B"   "PIP4K2A" "RNF220" 
 [8] "SLC44A1" "MBP"     "PLCL1"  

$OPC
 [1] "DSCAM"  "TNR"    "PTPRZ1" "PCDH15" "VCAN"   "KCNMB2" "GRID2"  "NRCAM" 
 [9] "NOVA1"  "AGAP1" 

$Pax6
[1] "MACROD2" "SGCD"    "ENOX1"   "PBX1"    "SORCS3"  "ROBO1"   "SOX2-OT"
[8] "GABRG3"  "CCK"    

$Pvalb
[1] "FGF12"   "SOX6"    "KCNC2"   "TENM2"   "ZNF804A" "GPC6"    "RPS6KA2"
[8] "MAGI1"  

$Sncg
[1] "ADAMTS9-AS2" "SORCS1"      "SGO1-AS1"    "PRKCA"       "RGS12"      
[6] "CNTNAP4"     "CNR1"       

$`Sst Chodl`
[1] "NPY"    "NOS1"   "LARGE1" "MPPED2" "SST"    "PAM"    "NELL1"  "PCDH7" 

$Sst
[1] "SPOCK3" "TRHDE"  "ASTN2" 

$Vip
[1] "CADM2"   "GALNT13" "L3MBTL4"

$VLMC
 [1] "DLC1"     "ATP1A2"   "PRKG1"    "NTRK3"    "EPS8"     "PARD3"   
 [7] "APOD"     "PDE8B"    "EBF1"     "MIR99AHG"

In [13]:
strsplit(file, "_")[[1]]

[1] "/home/xinyuelu/AD-proj/marker" "genes/ensembl/Astrocyte"      
[3] "markers.csv"

In [14]:
file

[1] "/home/xinyuelu/AD-proj/marker_genes/ensembl/Astrocyte_markers.csv"